In [29]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

In [6]:
train_file = pd.read_csv('./quora/train.csv')
test_file = pd.read_csv('./quora/test.csv')
sample_file = pd.read_csv('./quora/sample_submission.csv')

In [8]:
train_texts = train_file.question_text.tolist()
test_texts = test_file.question_text.tolist()
train_label = train_file.target.tolist()

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(train_texts)

In [15]:
x_train_seq = tokenizer.texts_to_sequences(train_texts)
x_text_seq = tokenizer.texts_to_sequences(test_texts)

In [17]:
x_train = tf.keras.preprocessing.sequence.pad_sequences(x_train_seq,maxlen=70)
x_text = tf.keras.preprocessing.sequence.pad_sequences(x_text_seq,maxlen=70)

In [41]:
import tensorflow.keras.backend as K

def val_loss(y_true, y_pred):
    return K.binary_crossentropy(y_true, y_pred)

def text_cnn(squence_length,vocab_size,embedding_size,l2_reg_lambda):
    #inputs
    text_seq = tf.keras.Input(shape=[squence_length])
    
    emb_text = layers.Embedding(vocab_size, embedding_size，weights=[embedMatrix],  # 表示直接使用预训练的词向量
                                trainable=False)(text_seq)
    
    convs = []
    
    filter_size = [2, 3, 4, 5]
    for fsz in filter_size:
        l_conv = layers.Conv1D(filters=100, kernel_size=fsz, activation='tanh')(emb_text)
        l_pool = layers.MaxPool1D(squence_length-fsz+1)(l_conv)
        l_pool = layers.Flatten()(l_pool)
        convs.append(l_pool)
    merge = layers.concatenate(convs, axis=1)
    
    out = layers.Dropout(0.5)(merge)
    output = layers.Dense(32, activation='relu')(out)
    
    output = layers.Dense(units=1, activation='sigmoid')(output)
    
    model = tf.keras.Model([text_seq], output)
    
    model.compile(loss='binary_crossentropy', optimizer="adam", metrics = ['acc'])
    return model
    
    

In [42]:
squence_length=70
vocab_size=2000
embedding_size=32
l2_reg_lambda=0.1

model = text_cnn(squence_length, vocab_size,embedding_size,l2_reg_lambda)
batch_size = 64
epochs = 10
history = model.fit(x_train, train_label, validation_split=0.1, batch_size = 64, epochs=epochs, shuffle = True)

Train on 1175509 samples, validate on 130613 samples
Epoch 1/10
 101760/1175509 [=>............................] - ETA: 7:07 - loss: 0.1567 - acc: 0.9441

KeyboardInterrupt: 